In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
from utils import *
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

%matplotlib inline

# Image Classification with Unstructured Environments

1. [Introduction and Motivation](#1.-Introduction-and-Motivation)
2. [General Techniques](#2.-General-Techniques)
3. Building a vocabulary
  1. Obtain training data
  2. Get SIFT feature vectors for all training images
  3. K-means clustering to get term vectors
4. Classifying images
  1. Load some test data, and test it
5. [Improving Feature Detection](#5.-Improving-Feature-Detection)
  2. [Saliency](#a.-Saliency)
  3. [Hue Information](#b.-Hue-Information)
6. Conclusion

## 1. Introduction and Motivation

In this mini problem set, you will how to apply image classification to marine environments. Unlike typical object recognition tasks that recognize objects that have limited variability in form, content, and shape, marine image classification requires a system that can handle incredible variability in object form. Take the following images of sand and seaweed, for example:

<img src="header.png" width="500" height="500"/>

These images include a tangle of objects, variable colors, occlusion, and other issues that make typical object recognition inaccurate. Throughout this mini problem set, you will learn how to build better models and methods for dealing with unstructured environments such as those found in marine habitats. The goal is to create a system with a higher accuracy in recognizing parts of a marine habitat, which would come in handy during AUV surveying and planning.

## 2. General Techniques

Before diving into applying the new techniques learned from lecture, let's explore state-of-the-art general techniques in image recognition. A prime example of this is provided from Google's Vision API. Visit their website at [https://cloud.google.com/vision/](https://cloud.google.com/vision/), and take a look at the services provided. Let's take a look what happens when we attempt to classify two different images.

First, save the image below of a horse to your computer, and upload it to the site above in the section labeled "Try the API". Note the words that it finds that are representative of the objects in that image.

<img src="horse.jpg" width="200" height="200"/>

Now, try doing the same thing with the below image of a seafloor. Once again, note the words that it decides are representative of the objects in the image.

<img src="sandy_image.png" width="200" height="200"/>

Given these results, answer the following questions:

**a) What words does it find for each of these images? What do you notice about the type of words that it finds for each image? Why do you think this is happening?**

**b) Why do you think it would be unfair to compare these two images using Google's system?**
<br/>
<div class="alert alert-info">
Please enter your answer in the following box.
</div>

INSERT ANSWER HERE

# 3. Building A Vocabulary

We have defined a function for you to use to obtain the training data for the Clustering Algorithm. This function iterates through every image file in the directory and SIFT runs on each image creating an array of feature vectors for each image. 

In the parent directory we have provided you with 2 folders containing images of auditoriums (labeled "Auditorium") and bowling alleys (labeled "Bowling"). 

The return_labels function takes in 5 inputs: 

* directoryList: list of directory labels
* range_size: the size of the term vector for each image, typically (added_limt * 128) if feature vector is 128
* added_limit: Given that not all images will not have the same number of interest points, we provide a smaller uniform number to consider. 
* count_limit: Desired number of interest points found in a single picture
* patch_size: The size of a patch(size of a feature around an interest point in pixels) defaulted to 32


Run the following cell to obtain the training data. You will need to find an appropriate count_limit and range_size. Remember range_size is (128*count_limit). You will find that return_labels does not work for high count_limits because  SIFT will not find enough feature points on an image.

In [18]:
sift_pictures = return_labels(['Bowling', 'Auditorium'], 2304, 18, 100)
print(sift_pictures)

18
('range: ', 2304)
('count: ', 100)
[[  6.   8.   2. ...   4.  18.  73.]
 [ 21.  12.  23. ...  30.  23.   6.]
 [ 14.  15.  44. ...   3.   2.   1.]
 ...
 [  4.   5.   6. ...  84. 101.  40.]
 [  8.   8.   7. ...  24.  29.  30.]
 [ 24.  27.  33. ...  38.  28.  29.]]


Analyze the dimensions of the output array from return_labels while considering the input arguments and explain the format of the output that it provides?
How would you split the data to process each feature independently?

<div class="alert alert-info">
Please answer in the following cell.
</div>

INSERT ANSWER HERE

Now that you have identified values to run return_labels and SIFT, what is the best values for range_size and count_limit to get the most accurate classification results. Run the following cell to find the clusters using K-Means clustering. 

In [3]:
count_limit = 18
range_size =  count_limit*128
labels_list = ['Bowling', 'Auditorium']
sift_pictures = return_labels(labels_list, range_size, count_limit, 100)
kmeans = KMeans(n_clusters=2, random_state=1).fit(sift_pictures)
print(sift_pictures)
preds = kmeans.predict(sift_pictures)
print(preds)

print(kmeans_accuracy(preds, labels_list))


18
('range: ', 2304)
('count: ', 100)
[[   0.    1.    1. ...,   14.   20.   19.]
 [  30.   29.   18. ...,    3.    1.    0.]
 [  39.  123.   51. ...,   50.   80.   53.]
 ..., 
 [  16.   29.   17. ...,    9.    1.    0.]
 [  68.   48.    1. ...,    1.    5.    8.]
 [   3.    3.    3. ...,   18.   29.   19.]]
[1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1
 0 0 0 0 1 1 0 0 0 0 1 1]
0.530612244898


Why do the values you found end up being the best values to run SIFT? How can you increase the accuracy?

<div class="alert alert-info">
Please answer in the following cell.
</div>

INSERT ANSWER HERE

# 4. Classifying Images

With the features that you found in the previous part, let's now implement a simple (yet powerful) classification algorithm.

Recall from lecture the K-nearest-neighbors (KNN) algorithm. Given a set of labeled data (in this case SIFT features corresponding to images) as "examples", the algorithm will classify a new datum based on the $k$ examples to which the unobserved datum is "nearest".

There are many ways to define distance between one example and another, but one of the most straightforward and common approaches is to use Euclidean distance (or "L2-norm" if you want to be fancy).

For this next part of the assignment, implement the KNN algorithm in the space provided. Then proceed to respond to the short-answer questions that follow.

In [5]:
def get_k_closest(point, known_points, k):
    '''
    point - unknown point (Numpy array of sift features)
    
    known_points - list of tuples describing labels of known points; 
                   first element of each tuple is the sift feature vector
                   second element of each tuple is the corresponding label
                   ex. [([...], 'label1'), ([...],'label2'),...]
                   
    k - an integer (corresponding to the number of neighbors to consider)
    
    Returns: the k tuples from known_points corresponding to the k closest points
             to the unknown point (as a list)
    '''
    
    raise NotImplementedError

In [ ]:
def knn(known_points, unknown_points, k):
    '''
    known_points - list of tuples describing labels of known points; 
                   first element of each tuple is the sift feature vector
                   second element of each tuple is the corresponding label
                   ex. [([...], 'label1'), ([...],'label2'),...]
                   
    unknown_points - list of sift feature vectors (Numpy arrays) corresponding to 
                    examples needing classification
                    
    k - an integer (corresponding to the number of neighbors to consider)
    
    Returns: list of tuples describing the assigned labels of previously unknown points
             (in the same order as the input)
    '''
    
    raise NotImplementedError

In [ ]:
#insert tests

Now answer the following questions about your implementation in the cell below.

1. Why don't we want to use even numbers as values of $k$?
2. How would you go about finding the best value of $k$?
3. What was the best value of $k$ that worked for the data given above? (TODO: figure out data to give)

<div class="alert alert-info">
Please enter your answer in the following box.
</div>

# 5. Improving Feature Detection

Now that we have gone through the process of obtaining feature vectors for our training images and classifying test images against this data, we can begin to make adjustments for handling unstructured environments. As covered in the lecture, two useful extensions are the use of saliency and hue information.

### a. Saliency
Recall from the lecture that saliency is a measure of entropy used to determine how "interesting" a patch is. We can use this to select regions of images that are better for training. Recall that the entropy for a discrete random variable is given by

$$E(\Delta I)=H=-\sum_{i=1}^{n}p(x_{i})\log p(x_{i})$$

For the gray scale image we define the probability $p(x)$ as follows,

$$p(x_{i})={h(x_{i\over N})={1\over N}\sum_{j=1}^{N}\delta(x_{i}-x_{j})}$$

where $\delta$ is the discrete Dirac delta function:
$$\delta(x)=\cases{1, x=0\cr 0, x\neq 0 } $$

So finally the entropy of a patch with $N_p$ pixels will be:
$$H_{g}=-{1\over N_{p}}\sum_{j=1}^{N_{p}}\log p_{g}(x_{j}) $$
It is important to note that we are calculating the log probability, $p_{g}$ using all the pixels in the image, but the outermost sum is just over the pixels in the patch.

Implement the function below which returns the total entropy for a given patch. The parameter `log_probs` is a list of values which represents the log probability of each pixel based off the global distribution of pixels.

<div class="alert alert-info">
Implement the method below.
</div>

In [18]:
def get_patch_entropy(log_probs):
    '''
    log_probs - list of values which represents the log probability of each pixel
    '''
    
    # YOUR CODE HERE
    raise NotImplementedError

In [19]:
def test_get_patch_entropy():
    test_patch_entropy_solution(get_patch_entropy)
    test_ok()
test_get_patch_entropy()

NotImplementedError: 

### b. Hue Information
As explained in the introduction, color is an important feature in correctly classifying unstructured environments such as coral. In order to incorporate this data into our training / features, we add the average hue of the patch to the feature vectors that are created using SIFT. The hue value can be visualized below, and is found by convering a pixel in RGB into the HSV color space.

<img src="hue.png" width="200" height="200"/>

If we have a 128-dimension feature vector from SIFT representing a patch, then the new feature vector would be of dimension 129. However, a trick/hack that we can employ is to copy the hue value into the feature vector multiple times. By adding, say, 10 more features which each hold the hue data, we may be able to achieve higher classification accuracies.

The graph below shows the change in classification accuracy as we change the number of times the hue feature has been added to the patch feature vector. The classification accuracy was found by classifying seaweed against sand.

<img src="hue_graph.png" width="500" height="500"/>

Given these results, answer the following questions:

**a) What do you notice about the effect of hue data on classifiction accuracy? Do you notice anything strange, or anything that was expected?**

**b) Why do you think that hue was a good feature to include, especially with these data sets?**

<div class="alert alert-info">
Please enter your answer in the following box.
</div>

PUT ANSWER HERE

## Done!

Congratulations, you've completed this problem set! Please save this notebook, validate it, and submit it.